# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [154]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
import scipy as sp


In [155]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


In [156]:
df.describe()

,admit,gre,gpa,prestige
count,397.000000,397.000000,397.000000,397.000000
mean,0.317380,587.858942,3.392242,2.488665
std,0.466044,115.717787,0.380208,0.947083
min,0.000000,220.000000,2.260000,1.000000
25%,0.000000,520.000000,3.130000,2.000000
50%,0.000000,580.000000,3.400000,2.000000
75%,1.000000,660.000000,3.670000,3.000000
max,1.000000,800.000000,4.000000,4.000000


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [157]:
# frequency table for prestige and whether or not someone was admitted
# Make a crosstab
my_tab1 = pd.crosstab(index=df["admit"],  columns="prestige")
my_tab1

col_0,prestige
admit,
0,271
1,126


In [158]:
#by each rank
my_tab1_1 = pd.crosstab(df["admit"],[df['prestige']]) 
                              
my_tab1_1

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [159]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix = 'prestige')

In [160]:
dummy_ranks.head()

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: We need one less than actual number of categories. For example, prestige has 4 categories between 1-4, therefore we need only 3 dummy variables.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [161]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1.0':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


/Users/AR/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [162]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted


my_tab2 = pd.crosstab(handCalc["admit"],handCalc['prestige_1.0'], margins=True)  # Make a crosstab
                              
my_tab2


prestige_1.0,0,1,All
admit,,,
0,243,28,271
1,93,33,126
All,336,61,397


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

odds = probability of X / probability against X.

P(X) = 33/61 = 0.54

P(against X) = 1-0.54 = 0.46

Odds(X) = 0.54/0.46 = 1.178

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

odds = probability of Y / probability against Y.

P(Y) = 93/336 = 0.276

P(against Y) = 1-0.276 = 0.723

Odds(Y) = 0.276/0.723 = 0.383
    

#### 3.3 Calculate the odds ratio

OR = odds (X)/ odds (Y)

OR = 1.178 / 0.383

OR = 3.079

#### 3.4 Write this finding in a sentenance: 

Taking the odds ratio as the percentage, there is a 300% higher chance of admission into grad school if an applicant attended the most prestigious undergraduate school.

#### 3.5 Print the cross tab for prestige_4

In [163]:
my_tab3 = pd.crosstab(handCalc["admit"],handCalc['prestige_4.0'], margins = "prestige") 
                              
my_tab3

prestige_4.0,0,1,All
admit,,,
0,216,55,271
1,114,12,126
All,330,67,397


#### 3.6 Calculate the OR 

x = admission into grad school from lowest ranking undergrad school.

y = admission into grad school from all other rankings of undergrad school.

P(x) = 12/67 = 0.179

P(against x) = 1 - 0.179 = 0.82

odds(x) = 0.218

P(y) = 114/330 = 0.345

P(against y) = 1 - 0.345 = 0.655

odds(y) = 0.527

OR = 0.4134

#### 3.7 Write this finding in a sentence

Answer: With attending the lowest ranking undergrad school, the odds of getting admitted are alot less than odds of getting rejected into grad school. They have decreased by 60% as shown by P(against y) or probability against getting into grad school while attending prestige_4.0 undergrad school.

## Part 4. Analysis

In [164]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [165]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [166]:
#slicing the admit column out, because that is  what we are prdicting based on prestige.
train_cols=['gre','gpa','prestige_2.0','prestige_3.0','prestige_4.0']
#or
train_cols = data.columns[1:]


#### 4.2 Fit the model

In [167]:
#using logit function
logit = sm.Logit(data['admit'], data[train_cols])
results = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [168]:
print results.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Tue, 29 Aug 2017   Pseudo R-squ.:                 0.08166
Time:                        16:04:32   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']
Comment: GRE has the smallest coefficient, maybe it has the least impact on admission? At least lesser impact than GPA.           
The coefficients are the Beta values, (the log odd scores). We are going to take exponential of those, to calculate the odds ratios.

In [169]:
#calculating the odds ratio of the coefficients.
#print np.exp(0.0022)
#print np.exp(0.7793)
#print np.exp(-0.6801)
#print np.exp(-1.3387)
#print np.exp(-1.5534)
#print np.exp(-3.8769)

# or 
print np.exp(results.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [170]:
#calculating the confidence interval
# o is 2.5% and 1 is 97.5% CI
print results.conf_int()

                     0         1
gre           0.000074  0.004362
gpa           0.127619  1.431056
prestige_2.0 -1.301337 -0.058936
prestige_3.0 -2.014579 -0.662776
prestige_4.0 -2.371624 -0.735197
intercept    -6.116077 -1.637631


#### 4.5 Interpret the OR of Prestige_2

Answer: The odds of getting into a grad school have decreased by 50% if the applicant is from a prestige_2.0 undergrad school.

#### 4.6 Interpret the OR of GPA

Answer: OR > 1 is associated with higher odds of outcome. Which can be translated into 200% higher chance of getting into grad school with higher numbers of GPA.
In contrast, GRE has OR approximately = 1, which means GRE scores do not have a noticeable impact on the admissions.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [171]:
#cartesian product is every possible combination with coordinate system. 
#we are interested in variables in certain ranges.
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)
# recursive cartesian, calling itself multiple times.
#its a bit like a loop, calling the same function with the answers from previous call.
# the values decrease in each call.
    m = n / arrays[0].size
    print(m)
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [172]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]
40
4
1
1


#### 5.1 Recreate the dummy variables

In [174]:
combos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
0    400 non-null float64
1    400 non-null float64
2    400 non-null float64
3    400 non-null float64
dtypes: float64(4)
memory usage: 12.6 KB


In [175]:
combos.describe()

,0,1,2,3
count,400.000000,400.000000,400.000000,400.0
mean,510.000000,3.130000,2.500000,1.0
std,185.334387,0.556003,1.119434,0.0
min,220.000000,2.260000,1.000000,1.0
25%,348.888889,2.646667,1.750000,1.0
50%,510.000000,3.130000,2.500000,1.0
75%,671.111111,3.613333,3.250000,1.0
max,800.000000,4.000000,4.000000,1.0


In [176]:
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
combos.describe()


,gre,gpa,prestige,intercept
count,400.000000,400.000000,400.000000,400.0
mean,510.000000,3.130000,2.500000,1.0
std,185.334387,0.556003,1.119434,0.0
min,220.000000,2.260000,1.000000,1.0
25%,348.888889,2.646667,1.750000,1.0
50%,510.000000,3.130000,2.500000,1.0
75%,671.111111,3.613333,3.250000,1.0
max,800.000000,4.000000,4.000000,1.0


In [177]:
# recreate the dummy variables
dummy_ranks2 = pd.get_dummies(combos['prestige'])


In [178]:
dummy_ranks2.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']
dummy_ranks2.head()

,prestige_1,prestige_2,prestige_3,prestige_4
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0


In [179]:
#data cleanup to keep relevant columns before prediction

cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])

#### 5.2 Make predictions on the enumerated dataset

In [180]:
#predict admission based on combos
#adding prediction column to the end of data frame.
combos['admit.predict'] = results.predict(combos[train_cols])

In [181]:
combos

,gre,gpa,prestige,intercept,prestige_2.0,prestige_3.0,prestige_4.0,admit.predict
0,220.0,2.260000,1.0,1.0,0.0,1.0,0.0,0.048977
1,220.0,2.260000,2.0,1.0,0.0,1.0,0.0,0.048977
2,220.0,2.260000,3.0,1.0,0.0,0.0,0.0,0.164173
3,220.0,2.260000,4.0,1.0,0.0,0.0,1.0,0.039890
4,220.0,2.453333,1.0,1.0,0.0,0.0,1.0,0.046078
5,220.0,2.453333,2.0,1.0,1.0,0.0,0.0,0.103682
6,220.0,2.453333,3.0,1.0,0.0,0.0,0.0,0.185907
7,220.0,2.453333,4.0,1.0,1.0,0.0,0.0,0.103682
8,220.0,2.646667,1.0,1.0,0.0,1.0,0.0,0.065080
9,220.0,2.646667,2.0,1.0,1.0,0.0,0.0,0.118543


#### 5.3 Interpret findings for the last 4 observations

Answer: By observing the admit.predict column and comparing those values to binary values in prestige columns, I have observed that any applicant from 4th ranking undergrad school has the lowest chance of getting admitted into grad school. Higher GPA (3.0) along with higher GRE scores, provide pretty good chances of admission, (if the students is coming from schools ranking 2-3). 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.

In [204]:
combos.head()


,gre,gpa,prestige,intercept,prestige_2.0,prestige_3.0,prestige_4.0,admit.predict
0,220.0,2.260000,1.0,1.0,0.0,1.0,0.0,0.048977
1,220.0,2.260000,2.0,1.0,0.0,1.0,0.0,0.048977
2,220.0,2.260000,3.0,1.0,0.0,0.0,0.0,0.164173
3,220.0,2.260000,4.0,1.0,0.0,0.0,1.0,0.039890
4,220.0,2.453333,1.0,1.0,0.0,0.0,1.0,0.046078


In [205]:
plot_col = ['prestige','prestige_2.0','prestige_3.0','prestige_4.0']

In [206]:
plot_col = pd.DataFrame(plot_col)

In [203]:
plot_col.head()

,0
0,prestige
1,prestige_2.0
2,prestige_3.0
3,prestige_4.0


I am not sure how to group the data by GRE and GPA from combos and plot only the column labels in plot_col.

Comment: A lot of help for parts 3 and beyond was taken from the following link:
        http://blog.yhat.com/posts/logistic-regression-and-python.html